In [ ]:
#虚拟机带不动 -- 
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

#制造一些随机的噪声来打破完全对称
def weight_variable(shape): 
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# tf.nn.conv2d是TensorFlow中的二维卷积函数
#W是卷积参数（前两个数字代表卷积核的尺寸，第三个数字表示有多少个通道，最后一个数字表示卷积核的数量）
#strides代表卷积模板移动的步长，第0位和第3为一定为1，中间两位表示横竖两个方向的步长
#SAME代表给边界加上padding（在边界自动补0）让卷积的输出和输入保持同样（SAME）尺寸
def conv2d(x, W): 
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 
                                                                    
#tf.nn.max_pool是TensorFlow中的最大池化函数
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder(tf.float32, [None, 784]) #特征
y_ = tf.placeholder(tf.float32, [None, 10]) #真实label
x_image = tf.reshape(x, [-1,28,28,1]) #前面的-1（Python里表示tensor的最后一个量，所以值对大值，有多少取多少）
                                      #代表样本数量不固定，最后1代表颜色通道数量

#第一个卷积层    
W_conv1 = weight_variable([5, 5, 1, 32]) #代表卷积核尺寸为5x5,1个颜色通道，32个不同的卷积核
b_conv1 = bias_variable([32]) 
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #卷积操作后加上偏置，接着再使用ReLU激活函数进行非线性处理
h_pool1 = max_pool_2x2(h_conv1)

#第二个卷积层
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#注意数值的计算
W_fc1 = weight_variable([7 * 7 * 64, 1024]) #tensor尺寸
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) #转换为一维向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#减轻过拟合
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#将Dropdout层的输出连接到一个Softmax层，得到概率输出
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#定义损失函数，优化器学习速率设为较小的1e-4
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#准确率
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#开始训练过程，初始化所有参数
tf.global_variables_initializer().run()
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_:batch[1], keep_prob:0.5})

#得到整体的分类准确率   
print("test accuracy %g"%accuracy.eval(feed_dict = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.0400
step 100, training accuracy 0.8400
step 200, training accuracy 0.9400
step 300, training accuracy 0.8600
step 400, training accuracy 0.9600
step 500, training accuracy 0.9200
step 600, training accuracy 0.9600
step 700, training accuracy 0.9800
step 800, training accuracy 0.9000
step 900, training accuracy 1.0000
